In [ ]:
from bs4 import BeautifulSoup
import requests
import urllib.parse
import csv
from datetime import datetime

def get_url(position,location,jobtype,dateposted):
    template = 'https://www.indeed.com/jobs?q={}&l={}&jt={}&fromage={}'
    url = template.format(position,location,jobtype,dateposted)
    return url

def get_record(card):
    atag = card.h2.a
    try:
        job_title = atag.get('title')
    except AttributeError:
        job_title = ''
    job_url = 'https://www.indeed.com' + atag.get('href')
    try:
        company = card.find('span','company').text.strip()
    except AttributeError:
        company = ''
    job_location = card.find('div','recJobLoc').get('data-rc-loc')
    job_summary = card.find('div','summary').text.strip()
    post_date = card.find('span','date').text
    today = datetime.today().strftime('%m-%d-%Y')
    try:
        job_salary = card.find('span','salaryText').text.strip()
    except AttributeError:
        job_salary = ""
    
    record = (job_title, company, job_location, post_date, today, job_summary, job_salary, job_url)
    
    return record

def main(position,location,jobtype,dateposted):
    records = []
    url = get_url(position,location,jobtype,dateposted)
    
    while True:
        response = requests.get(url)
        soup = BeautifulSoup(response.text,'html.parser')
        cards = soup.find_all('div','jobsearch-SerpJobCard')
        
        for card in cards:
            record = get_record(card)
            records.append(record)
        try:
            url = 'https://www.indeed.com' + soup.find('a',{"aria-label":"Next"}).get('href')
        except AttributeError:
            break
    #Replace file name every time to save new records        
    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Title', 'Company', 'Location', 'PostDate', 'ExtractDate', 'Summary', 'Salary', 'JobUrl'])
        writer.writerows(records)

In [ ]:
#Field 1: Position
#Field 2: Location
#Field 3: Full Time / Internship
#Field 4: 1 = Last 24 hours, 3 = Last 3 days, 7 = Last 7 days, 14 = Last 14 days, last = Since your last visit
main('data science','united states','internship','1')